# Dependencias

In [30]:
import cv2
import numpy as np
from pathlib import Path
import random
import shutil
import yaml

# Directorios y configuración

In [31]:
SRC_DIR = (Path("..") / "Imagenes_defectos").resolve()
DST_DIR = (Path("..") / "Deteccion" / "data").resolve()

TRAIN_RATIO = 0.85              # deja más train
VAL_MIN_POS = 30                # mínimo defectos en val (ajusta)
AUG_PER_POS = 8                 # augmentaciones por imagen con defecto (train)
CLASS_ID = 0
THRESH_WHITE = 200              # tu umbral para defecto en máscara (ajusta si hiciera falta)

random.seed(42)

# Limpiamos las carpetas
if DST_DIR.exists():
    shutil.rmtree(DST_DIR)

for split in ["train", "val"]:
    (DST_DIR / "images" / split).mkdir(parents=True, exist_ok=True)
    (DST_DIR / "labels" / split).mkdir(parents=True, exist_ok=True)

# Pasar de mask a box
def mask_to_bboxes_from_array(mask: np.ndarray, img_shape):
    binary = cv2.inRange(mask, THRESH_WHITE, 255)
    kernel = np.ones((3, 3), np.uint8)
    binary = cv2.dilate(binary, kernel, iterations=1)

    num_labels, _, stats, _ = cv2.connectedComponentsWithStats(binary)
    h_img, w_img = img_shape[:2]

    boxes = []
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area < 5:
            continue
        xc = (x + w / 2) / w_img
        yc = (y + h / 2) / h_img
        bw = w / w_img
        bh = h / h_img
        boxes.append((xc, yc, bw, bh))
    return boxes

# Data augmentation
def augment_pair(img, mask):
    h, w = img.shape[:2]
    angle = random.uniform(-7, 7)
    tx = random.uniform(-0.04, 0.04) * w
    ty = random.uniform(-0.04, 0.04) * h
    scale = random.uniform(0.97, 1.03)

    M = cv2.getRotationMatrix2D((w/2, h/2), angle, scale)
    M[0, 2] += tx
    M[1, 2] += ty

    img2 = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    m2   = cv2.warpAffine(mask, M, (w, h), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_REFLECT)

    if random.random() < 0.5:
        img2 = cv2.flip(img2, 1)
        m2   = cv2.flip(m2, 1)

    # blur leve opcional
    if random.random() < 0.25:
        img2 = cv2.GaussianBlur(img2, (3, 3), 0)

    return img2, m2

# Con fallo y si fallo
pairs_pos = []
pairs_neg = []

for kos_dir in sorted(SRC_DIR.glob("kos*")):
    if not kos_dir.is_dir():
        continue
    for img_path in sorted(list(kos_dir.glob("Part*.jpg")) + list(kos_dir.glob("Part*.JPG"))):
        mask_path = kos_dir / f"{img_path.stem}_label.bmp"
        if not mask_path.exists():
            continue

        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        if mask is None:
            continue

        # Si hay algún pixel blanco, lo consideramos "con defecto"
        if mask.max() >= THRESH_WHITE:
            pairs_pos.append((img_path, mask_path))
        else:
            pairs_neg.append((img_path, mask_path))

print("Pares con defecto:", len(pairs_pos))
print("Pares sin defecto:", len(pairs_neg))

if len(pairs_pos) < 5:
    raise RuntimeError("Demasiado pocos defectos reales. Revisa THRESH_WHITE o tus máscaras.")

random.shuffle(pairs_pos)
random.shuffle(pairs_neg)

# En val tiene que haber defectos si o si
val_pos = pairs_pos[:max(VAL_MIN_POS, int(len(pairs_pos) * (1 - TRAIN_RATIO)))]
train_pos = pairs_pos[len(val_pos):]

# Negativos: mantenemos proporción
n_train_neg = min(len(pairs_neg), max(len(train_pos) * 2, 200))  # 2x pos o 200
n_val_neg   = min(len(pairs_neg) - n_train_neg, max(len(val_pos), 50))

train_neg = pairs_neg[:n_train_neg]
val_neg   = pairs_neg[n_train_neg:n_train_neg + n_val_neg]

print("Train pos:", len(train_pos), "Train neg:", len(train_neg))
print("Val   pos:", len(val_pos),   "Val   neg:", len(val_neg))

# Guardar muestra (img + label)
def save_yolo_sample(img, mask, split, name):
    boxes = mask_to_bboxes_from_array(mask, img.shape)

    out_img = DST_DIR / "images" / split / f"{name}.jpg"
    out_lbl = DST_DIR / "labels" / split / f"{name}.txt"

    cv2.imwrite(str(out_img), img)
    with open(out_lbl, "w") as f:
        for xc, yc, bw, bh in boxes:
            f.write(f"{CLASS_ID} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

# Generar Train (pos + aug) y val (sin aug)
def process_pairs(pairs, split, do_aug=False, aug_per=0):
    for img_path, mask_path in pairs:
        img = cv2.imread(str(img_path))
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        if img is None or mask is None:
            continue

        base = f"{img_path.parent.name}_{img_path.stem}"
        save_yolo_sample(img, mask, split, base)

        if do_aug:
            for k in range(aug_per):
                img2, m2 = augment_pair(img, mask)
                save_yolo_sample(img2, m2, split, f"{base}_aug{k+1}")

# Train: pos con augment fuerte, neg sin augment
process_pairs(train_pos, "train", do_aug=True, aug_per=AUG_PER_POS)
process_pairs(train_neg, "train", do_aug=False)

# Val: sin augment (importante para evaluar)
process_pairs(val_pos, "val", do_aug=False)
process_pairs(val_neg, "val", do_aug=False)

# data.yaml
data_yaml = {
    "train": str(DST_DIR / "images" / "train"),
    "val": str(DST_DIR / "images" / "val"),
    "nc": 1,
    "names": ["defecto"],
}
(DST_DIR / "data.yaml").write_text(yaml.safe_dump(data_yaml, sort_keys=False), encoding="utf-8")

# Resumen para saber si funciona
def count_nonempty_labels(split):
    label_dir = DST_DIR / "labels" / split
    n_total = 0
    n_pos = 0
    for p in label_dir.glob("*.txt"):
        n_total += 1
        if p.stat().st_size > 0:
            n_pos += 1
    return n_total, n_pos

train_total, train_pos_count = count_nonempty_labels("train")
val_total, val_pos_count = count_nonempty_labels("val")

print("\n Dataset creado en:", DST_DIR)
print(f"Train labels: {train_total} (con defecto: {train_pos_count})")
print(f"Val   labels: {val_total} (con defecto: {val_pos_count})")


Pares con defecto: 86
Pares sin defecto: 512
Train pos: 56 Train neg: 200
Val   pos: 30 Val   neg: 50

 Dataset creado en: C:\Users\User\Documents\GitHub\grupo1reto2\Deteccion\data
Train labels: 513 (con defecto: 351)
Val   labels: 74 (con defecto: 26)


# Prueba modelo baseline

In [32]:
from ultralytics import YOLO

#model = YOLO("yolov8n.pt")
#model.train(
 #   data="data/data.yaml",
  #  device="cpu",
   # imgsz=320,
    #batch=2,
    #epochs=30,
#    workers=0,
 #   mosaic=0.0,
  #  mixup=0.0,
   # project="runs",
    #name="yolo_cpu_v2_aug"
#)

In [33]:
# Cargar modelo entrenado
model = YOLO("runs/yolo_cpu_v2_aug/weights/best.pt")
# ajusta la ruta si tu entrenamiento final está en otro sitio

# Imagen de prueba (una con defecto)
img_path = Path("..") / "Imagenes_defectos" / "kos01" / "Part5.jpg"

# Inferencia
results = model.predict(
    source=str(img_path),
    conf=0.3,   # bajo para ver si detecta algo
    iou=0.3,
    device="cpu",
    verbose=False
)

# Ver resultados en consola
r = results[0]
print("Número de detecciones:", 0 if r.boxes is None else len(r.boxes))

# Dibujar y mostrar
img = cv2.imread(str(img_path))
if r.boxes is not None:
    for b in r.boxes:
        x1, y1, x2, y2 = map(int, b.xyxy[0])
        conf = float(b.conf[0])
        cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(img, f"{conf:.2f}", (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

cv2.imshow("deteccion", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Número de detecciones: 1


In [34]:
model = YOLO("yolo11s.pt")
model.train(
    data="data/data.yaml",
    device="cpu",
    imgsz=320,
    batch=2,
    epochs=30,
    workers=0,
    mosaic=0.0,
    mixup=0.0,
    project="runs",
    name="yolo_cpu_v3_aug"
)

New https://pypi.org/project/ultralytics/8.4.2 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.252  Python-3.13.5 torch-2.9.1+cpu CPU (Intel Core(TM) Ultra 7 165H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolo_cpu_v3_aug2, nbs=64, nms=False, opset=None, opti

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020D22372F90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480